In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import datetime
import seaborn as sns
import keras

Using TensorFlow backend.


In [2]:
shops = pd.read_csv("data/shops.csv")
items = pd.read_csv("data/items.csv")
item_categories = pd.read_csv("data/item_categories.csv")
test = pd.read_csv("data/test.csv")
sales_train = pd.read_csv("data/sales_train.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [3]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
#now we will create a pivot tabel by going so we get our data in desired form 
#we want get total count value of an item over the whole month for a shop 
# That why we made shop_id and item_id our indices and date_block_num our column 
# the value we want is item_cnt_day and used sum as aggregating function 
dataset = sales_train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [6]:
dataset.head()

item_cnt_day                             ...                 \
date_block_num            0   1  2  3  4  5  6  7  8  9  ... 24 25 26 27 28   
shop_id item_id                                          ...                  
0       30                 0  31  0  0  0  0  0  0  0  0 ...  0  0  0  0  0   
        31                 0  11  0  0  0  0  0  0  0  0 ...  0  0  0  0  0   
        32                 6  10  0  0  0  0  0  0  0  0 ...  0  0  0  0  0   
        33                 3   3  0  0  0  0  0  0  0  0 ...  0  0  0  0  0   
        35                 1  14  0  0  0  0  0  0  0  0 ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
0       30       0  0  0  0  0  
        31       0  0  0  0  0  
        32       0  0  0  0  0  
        33       0  0  0  0  0  
        35       0  0  0  0  0  

[5 rows x 34 columns]

In [7]:
# lets reset our indices, so that data should be in way we can easily manipulate
dataset.reset_index(inplace = True)

In [8]:
# lets check on our pivot table
dataset.head()

shop_id item_id item_cnt_day                       ...        \
date_block_num                            0   1  2  3  4  5  6  7 ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0 ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0 ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0 ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0 ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0 ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  

[5 rows x 36 columns]

In [9]:
# Now we will merge our pivot table with the test_data because we want to keep the data of items we have
# predict
dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning:

merging between different levels can give an unintended result (1 levels on the left, 2 on the right)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [10]:
# lets fill all NaN values with 0
# dataset.fillna(0,inplace = True)
dataset.fillna(dataset.mean(), inplace=True)
# lets check our data now 
dataset.head()

,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000,3.000000,1.000000,0.000000
1,1,5,5320,0.18502,0.198943,0.283643,0.185846,0.209346,0.291776,0.268931,...,0.606585,0.490081,0.492379,0.529128,0.50605,0.464992,0.469507,0.534783,0.528024,0.564154
2,2,5,5233,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.00000,2.000000,0.000000,1.000000,3.000000,1.000000
3,3,5,5232,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000
4,4,5,5268,0.18502,0.198943,0.283643,0.185846,0.209346,0.291776,0.268931,...,0.606585,0.490081,0.492379,0.529128,0.50605,0.464992,0.469507,0.534783,0.528024,0.564154


In [11]:
# we will drop shop_id and item_id because we do not need them
# we are teaching our model how to generate the next sequence 
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000,3.000000,1.000000,0.000000
1,0.18502,0.198943,0.283643,0.185846,0.209346,0.291776,0.268931,0.290331,0.395874,0.356836,...,0.606585,0.490081,0.492379,0.529128,0.50605,0.464992,0.469507,0.534783,0.528024,0.564154
2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.00000,2.000000,0.000000,1.000000,3.000000,1.000000
3,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000
4,0.18502,0.198943,0.283643,0.185846,0.209346,0.291776,0.268931,0.290331,0.395874,0.356836,...,0.606585,0.490081,0.492379,0.529128,0.50605,0.464992,0.469507,0.534783,0.528024,0.564154


In [12]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33, 1) (214200, 1) (214200, 33, 1)


In [13]:
# importing libraries required for our model
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [23]:
# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1),  return_sequences=True))

my_model.add(LSTM(32, activation='relu', return_sequences=True))
my_model.add(LSTM(16, activation='relu', return_sequences=True))
my_model.add(LSTM(1, activation='relu'))
my_model.add(Dropout(0.4))
my_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
my_model.add(Dense(10, kernel_initializer='glorot_normal', activation='relu'))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 33, 64)            16896     
_________________________________________________________________
lstm_24 (LSTM)               (None, 33, 32)            12416     
_________________________________________________________________
lstm_25 (LSTM)               (None, 33, 16)            3136      
_________________________________________________________________
lstm_26 (LSTM)               (None, 1)                 72        
_________________________________________________________________
dropout_5 (Dropout)          (None, 1)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                20        
_________________________________________________________________
dense_14 (Dense)             (None, 10)              

In [24]:
my_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)

Epoch 1/10
214200/214200 [==============================] - 69s 321us/step - loss: 30.9546 - mean_squared_error: 30.9546
Epoch 2/10
214200/214200 [==============================] - 93s 433us/step - loss: 30.6995 - mean_squared_error: 30.6995
Epoch 3/10
214200/214200 [==============================] - 79s 369us/step - loss: 30.6880 - mean_squared_error: 30.6880
Epoch 4/10
214200/214200 [==============================] - 73s 339us/step - loss: 30.5528 - mean_squared_error: 30.5528
Epoch 5/10
214200/214200 [==============================] - 81s 380us/step - loss: 30.7346 - mean_squared_error: 30.7346
Epoch 6/10
214200/214200 [==============================] - 78s 365us/step - loss: 30.5909 - mean_squared_error: 30.5909
Epoch 7/10
214200/214200 [==============================] - 71s 331us/step - loss: 30.6681 - mean_squared_error: 30.6681
Epoch 8/10
214200/214200 [==============================] - 72s 336us/step - loss: 29.9933 - mean_squared_error: 29.9933
Epoch 9/10
214200/214200 [======

In [27]:
# creating submission file 
submission_pfs = my_model.predict(X_test)
# we will keep every value between 0 and 20
submission_pfs = submission_pfs.clip(0,20)
# creating dataframe with required columns 
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
submission.to_csv('sub_pfs.csv',index = False)